<a href="https://colab.research.google.com/github/zonezero2604/Hkr_Set/blob/master/Bai_16_save_and_load_vn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Save and load models

Tiến trình của mô hình có thể được lưu trong — và sau — quá trình đào tạo. Điều này có nghĩa là một mô hình có thể tiếp tục lại khi nó đã dừng lại và tránh thời gian đào tạo dài. Lưu cũng có nghĩa là bạn có thể chia sẻ mô hình của mình và những người khác có thể tạo lại tác phẩm của bạn. Khi xuất bản các mô hình và kỹ thuật nghiên cứu, hầu hết các học viên máy học chia sẻ:

* code để tạo mô hình và
* Weight hoặc thông số được đào tạo cho mô hình (hyperparametters)

Chia sẻ dữ liệu này giúp những người khác hiểu cách hoạt động của mô hình và tự mình thử với dữ liệu mới.

Thận trọng: Hãy cẩn thận với mã không đáng tin cậy — Các mô hình TensorFlow là mã. Xem [Sử dụng TensorFlow an toàn](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) để biết chi tiết.

### Tùy chọn

Có nhiều cách khác nhau để lưu các mô hình TensorFlow — tùy thuộc vào API bạn đang sử dụng. Hướng dẫn này sử dụng [tf.keras](https://www.tensorflow.org/guide/keras), một API cấp cao để xây dựng và đào tạo các mô hình trong TensorFlow. Đối với các phương pháp tiếp cận khác, hãy xem hướng dẫn TensorFlow [Lưu và khôi phục](https://www.tensorflow.org/guide/saved_model) hoặc [lưu trong eager](https://www.tensorflow.org/guide/eager#object-based_saving).


## Setup

### Installs and imports

cài đặt và import TensorFlow và dependencies (các phụ thuộc):

In [1]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [2]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


### Lấy một tập dữ liệu mẫu

Để mô phỏng các save-load weight, bạn sẽ sử dụng [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Để tăng tốc và chạy khỏng 1000 samples đầu tiên:

In [3]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]  # chỉ lấy 1 nghìn mẫu để train (ko cần nhiều)
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images  = test_images[:1000].reshape(-1, 28 * 28) / 255.0

11493376/11490434 [==============================] - 0s 0us/step


### Định nghĩa một model

Bắt đầu bằng cách xây dựng một mô hình tuần tự đơn giản:

In [4]:
# Define a simple sequential model
def create_model():
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


## Lưu checkpoints trong quá trình training

Bạn có thể sử dụng một mô hình đã được đào tạo mà không cần phải đào tạo lại hoặc tiếp tục đào tạo từ nơi bạn đã dừng lại — trong trường hợp quá trình đào tạo bị gián đoạn.
Hàm `tf.keras.callbacks.ModelCheckpoint` cho phép bạn lưu mô hình trong và sau quá trình tranning

### Sử dụng Checkpoint callback 

Tạo một hàm `tf.keras.callbacks.ModelCheckpoint` để lưu thông tin training:

In [5]:
# //OK 1:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Epoch 1/10
32/32 [==============================] - 4s 16ms/step - loss: 1.6238 - accuracy: 0.5164 - val_loss: 0.7406 - val_accuracy: 0.7760

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4749 - accuracy: 0.8813 - val_loss: 0.5389 - val_accuracy: 0.8310

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.3011 - accuracy: 0.9210 - val_loss: 0.4858 - val_accuracy: 0.8440

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2096 - accuracy: 0.9616 - val_loss: 0.4680 - val_accuracy: 0.8540

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1334 - accuracy: 0.9760 - val_loss: 0.4627 - val_accuracy: 0.8540

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

Điều này tạo ra một bộ sưu tập các tệp  TensorFlow checkpoint được cập nhật vào cuối mỗi epoch:

In [6]:
!ls -la -h {checkpoint_dir}

total 4.7M
drwxr-xr-x 2 root root 4.0K May 24 08:17 .
drwxr-xr-x 1 root root 4.0K May 24 08:17 ..
-rw-r--r-- 1 root root   71 May 24 08:17 checkpoint
-rw-r--r-- 1 root root 4.7M May 24 08:17 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root 1.2K May 24 08:17 cp.ckpt.index


Tạo một mô hình mới, chưa qua đào tạo. Khi khôi phục một mô hình từ weight đã lưu, bạn phải có một mô hình có cùng *kiến trúc* với mô hình ban đầu. Khi hai kiến trúc tương đồng, bạn có thể dùng chung weight của các thời điểm training khác nhau.

Bây giờ xây dựng tiếp một mô hình mới, chưa được đào tạo và đánh giá nó trên bộ thử nghiệm. Một mô hình chưa được đào tạo sẽ hoạt động ở mức cơ hội (độ chính xác ~ 10%):

In [7]:
# Create a basic model instance
model1 = create_model()

# Evaluate the model
loss, acc = model1.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3301 - accuracy: 0.0900
Untrained model, accuracy:  9.00%


Sau đó load weights từ các file checkpoint và đánh giá lại tập test:

In [8]:
# //OK 1: load
# Loads the weights
model1.load_weights(checkpoint_path)

# Re-evaluate the model
loss,acc = model1.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4199 - accuracy: 0.8690
Restored model, accuracy: 86.90%


### Các tùy chọn Checkpoint callback 

Checkpoint cho phép đặt các tùy chọn sao cho tên file sẽ được lưu là duy nhất cho mỗi lần lưu trữ, điều này giúp cho chúng ta chọn được weight có giá trị tốt nhất sau quá trình training.

In [9]:
!rm /content/training_2/*
!rm -d /content/training_2


rm: cannot remove '/content/training_2/*': No such file or directory
rm: cannot remove '/content/training_2': No such file or directory


In [10]:
tf.keras.callbacks.ModelCheckpoint??

In [11]:
# OK 2: save
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_best_only=True
    )

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
        #   steps_per_epoch=1,
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00001: val_loss improved from inf to 0.73543, saving model to training_2/cp.ckpt

Epoch 00002: val_loss improved from 0.73543 to 0.55148, saving model to training_2/cp.ckpt

Epoch 00003: val_loss improved from 0.55148 to 0.50379, saving model to training_2/cp.ckpt

Epoch 00004: val_loss improved from 0.50379 to 0.44122, saving model to training_2/cp.ckpt

Epoch 00005: val_loss improved from 0.44122 to 0.41914, saving model to training_2/cp.ckpt

Epoch 00006: val_loss improved from 0.41914 to 0.41833, saving model to training_2/cp.ckpt

Epoch 00007: val_loss did not improve from 0.41833

Epoch 00008: val_loss improved from 0.41833 to 0.41166, saving model to training_2/cp.ckpt

Epoch 00009: val_loss did not improve from 0.41166

Epoch 00010: val_loss improved from 0.41166 to 0.40388, saving model to training_2/cp.ckpt

Epoch 00011: val_loss did not improve from 0.40388

Epoch 00012: val_loss improved from 0.40388 to 0.40345, saving model to training_2/cp.ckpt

Epoch 00013: val_lo

Bây giờ, hãy xem các checkpoint và chọn checkpoint mới nhất:

In [12]:
!ls -la -h {checkpoint_dir}

total 4.7M
drwxr-xr-x 2 root root 4.0K May 24 08:17 .
drwxr-xr-x 1 root root 4.0K May 24 08:17 ..
-rw-r--r-- 1 root root   71 May 24 08:17 checkpoint
-rw-r--r-- 1 root root 4.7M May 24 08:17 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 root root 1.2K May 24 08:17 cp.ckpt.index


In [13]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp.ckpt'

Lưu ý: định dạng tensorflow mặc định chỉ lưu 5 checkpoints gần nhất.
Để test, hãy reset model và thực hiện dự đoán:

In [14]:
# OK 2: Load
# Create a new model instance
model2 = create_model()

# Load the previously saved weights
model2.load_weights(latest)

# Re-evaluate the model
loss, acc = model2.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3958 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Những tệp này là gì?

Các đoạn code trên lưu các weight vào các tệp [checkpoint](https://www.tensorflow.org/guide/saved_model#save_and_restore_variables)-các file checkpoint chứa các thông tin:
* Một hoặc nhiều phân đoạn chứa trọng số của mô hình.
* một file index chỉ ra weights nào được lưu trữ trong phân đoạn nào.

Nếu bạn chỉ đào tạo một mô hình trên một máy duy nhất, bạn sẽ có một phân đoạn với hậu tố: `.data-00000-of-00001`

## Lưu trọng số theo cách thủ công

Lưu trọng số bằng cách dùng lệnh `Model.save_weights`. Mặc định, `tf.keras`— và `save_weights` sử dụng checkpoint với đuôi `.ckpt`.

In [15]:
# OK 3: Save & Load
# Save the weights
model2.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model3 = create_model()

# Restore the weights
model3.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss,acc = model3.evaluate(test_images,  test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.3958 - accuracy: 0.8750
Restored model, accuracy: 87.50%


## Lưu toàn bộ mô hình (cả model+weight)



In [16]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy
import os
from os.path import exists
# fix random seed for reproducibility
numpy.random.seed(211)
# load pima indians dataset
Pdata='pima-indians-diabetes.data.csv'
if not exists(Pdata):
    !wget https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv

dataset = numpy.loadtxt(Pdata, delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]
# create model
model = Sequential()
model.add(Dense(12, input_dim=8, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, epochs=150, batch_size=10, verbose=0)
# evaluate the model
scores = model.evaluate(X, Y, verbose=0)
print("1st: %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
 
# serialize model to JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")
 
# later...
 
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
score = loaded_model.evaluate(X, Y, verbose=0)
print("2nd: %s: %.2f%%" % (loaded_model.metrics_names[1], score[1]*100))

--2021-05-24 08:17:36--  https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23278 (23K) [text/plain]
Saving to: ‘pima-indians-diabetes.data.csv’

pima-indians-diabet 100%[===================>]  22.73K  --.-KB/s    in 0s      

2021-05-24 08:17:36 (125 MB/s) - ‘pima-indians-diabetes.data.csv’ saved [23278/23278]

1st: accuracy: 78.39%
Saved model to disk
Loaded model from disk
2nd: accuracy: 78.39%


In [17]:
X

array([[  6.   , 148.   ,  72.   , ...,  33.6  ,   0.627,  50.   ],
       [  1.   ,  85.   ,  66.   , ...,  26.6  ,   0.351,  31.   ],
       [  8.   , 183.   ,  64.   , ...,  23.3  ,   0.672,  32.   ],
       ...,
       [  5.   , 121.   ,  72.   , ...,  26.2  ,   0.245,  30.   ],
       [  1.   , 126.   ,  60.   , ...,  30.1  ,   0.349,  47.   ],
       [  1.   ,  93.   ,  70.   , ...,  30.4  ,   0.315,  23.   ]])

In [18]:
loaded_model_json

'{"class_name": "Sequential", "config": {"name": "sequential_5", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [null, 8], "dtype": "float32", "sparse": false, "ragged": false, "name": "dense_10_input"}}, {"class_name": "Dense", "config": {"name": "dense_10", "trainable": true, "batch_input_shape": [null, 8], "dtype": "float32", "units": 12, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}}, {"class_name": "Dense", "config": {"name": "dense_11", "trainable": true, "dtype": "float32", "units": 8, "activation": "relu", "use_bias": true, "kernel_initializer": {"class_name": "GlorotUniform", "config": {"seed": null}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null,